In [10]:
import redis
from mlrepricer import redisdb, helper, parser, setup, updateprice
import xmltodict

In [2]:
r = redis.StrictRedis(**helper.rediscred, decode_responses=True)
redisdb.server_start()

Example iterate over all keys

In [3]:
def asin_generator():
    """
    Match keys which contain all messages per asin.
    Key example: B07B9NGM2J
    Return a generator of keys you can use to get actual messages.
    """
    return r.scan_iter(match=r'[a-zA-Z0-9]'*10)

In [4]:
def actionable_asins():
    """
    Get asin we have to take actions.
    one key: updated_asins
    """
    return r.sscan_iter('updated_asins')

In [5]:
def parsed_messages(asin):
    """Make a list with all messages parsed."""
    mlist = r.zrevrangebyscore(asin, 'inf', '-inf')
    return [parser.parse(eval(message)) for message in mlist]

In [6]:
#[x for x in message_key()]

In [7]:
result = []
asin_gen = asin_generator()
for asin in asin_gen:
    result.append(parsed_messages(asin))

In [8]:
result[33][1]

,asin,feedback,feedbackpercent,instock,isbuyboxwinner,isfeaturedmerchant,isprime,price,sellerid,shipping_maxhours,shipping_minhours,time_changed
0,B075NK775D,1640,98,1,1,1,0,13.70,A2EHWGAW6J9W8Q,48,24,2018-07-28 22:33:20.982
1,B075NK775D,71,86,1,0,1,0,13.71,A2UAZ7ZIKEWIZM,48,24,2018-07-28 22:33:20.982
2,B075NK775D,46,100,1,0,1,0,13.90,A1S6AD6PHV3V62,48,24,2018-07-28 22:33:20.982


In [11]:
updateprice.get_latest_message(asin)

TypeError: getaddrinfo() argument 1 must be string or None